In [1]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import json
import sys
sys.path.append('../')
print(sys.path)
import prompts as p




# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DATA_PATH = "../../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row.csv"
OUTPUT_PATH = "../../../Datasets/Evaluations/Sentiment Analysis/Sentiment_Analysis_Zero_Shot_Multi_Term_3.5_instruct.csv"

MODEL = "gpt-3.5-turbo-instruct"
TEMP = 0



['/Users/timgutberlet/My Drive (t.gutberlet01@gmail.com)/05 - Coding/06 - Bachelor Thesis/Error-Analysis-with-LLMs/Sentiment Analysis/Pipelines/multi_term', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/timgutberlet/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '../']


In [2]:
df = pd.read_csv(DATA_PATH)
prompt_name = "prompt_multi_term_zeroshot"
prompt = p.prompt_multi_term_zeroshot


df['polarity_pred'] = None
df['prompt_name'] = None
df['prompt'] = None
print(df.head())


   text_id  term_id                                               text  \
0        0        0  Amusing details distinguish desserts, from dul...   
1        0        1  Amusing details distinguish desserts, from dul...   
2        0        2  Amusing details distinguish desserts, from dul...   
3        0        3  Amusing details distinguish desserts, from dul...   
4        1        4  The server was so busy the night we visited th...   

                          term  polarity polarity_pred prompt_name prompt  
0                     desserts  positive          None        None   None  
1     dulce de leche ice-cream   neutral          None        None   None  
2  chocolate sauce tic-tac-toe   neutral          None        None   None  
3            poached pineapple   neutral          None        None   None  
4                       server  negative          None        None   None  


In [3]:
def create_terms(df):
    s = ""
    for term in df['term']:
        s += "'"+ term + "'" + "\n"
    return s

In [4]:
def execute(df, prompt, prompt_name, start = 0):
    current_text = -1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    for i in range(start, len(df)):
        if df["text_id"][i] == current_text:
            continue
        current_text = df["text_id"][i]
        if(df["polarity_pred"][i] != None and df["polarity_pred"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        input_text = df["text"][i]
        terms_df = df[df["text_id"] == current_text]
        terms = create_terms(terms_df)
        try: 
            result = chain.run(input_text = input_text, terms = terms, callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'polarity_pred'] =  'error'
            df.loc[i, 'prompt_name'] = prompt_name
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(input_text = input_text, terms = terms).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(input_text = input_text, terms = terms).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        data = json.loads(result)
        term_sentiments= data["term_sentiments"]
        x = 0
        for j in terms_df["term_id"]:
            if df["term"][j].lower() == term_sentiments[x][0].lower():
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            else:
                print(df["term"][j].lower())
                print(term_sentiments[x][0].lower())
                print(prompt_text)
                print("\n")
                print(result)
                print("\n")
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            x+=1
    return df

df = execute(df, prompt, prompt_name)
    



0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Task: Analyze the sentiment of specific terms mentioned in a sentence. 
You are required to evaluate whether the sentiment towards each term is 'positive', 'negative', or 'neutral'.

Sentence: "Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple."

Terms:
'desserts'
'dulce de leche ice-cream'
'chocolate sauce tic-tac-toe'
'poached pineapple'


Return the final result as JSON in the format {"term_sentiments": "<a list of [term, sentiment] pairs>"}.
Always write a term like it is written in the text and do NOT return anything else. 
Answer:


 {"term_sentiments": [["desserts", "neutral"], ["dulce de leche ice-cream", "positive"], ["chocolate sauce tic-tac-toe", "neutral"], ["poached pineapple", "neutral"]]}


0 of  451
4
Task: Analyze the sentiment of specific terms mentioned in a sentence. 
You are required to evaluate whether the sen

In [5]:
error_rows = df[df['polarity_pred'] == 'error']
print(error_rows)


Empty DataFrame
Columns: [text_id, term_id, text, term, polarity, polarity_pred, prompt_name, prompt]
Index: []


In [6]:
df.head()

,text_id,term_id,text,term,polarity,polarity_pred,prompt_name,prompt
0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,neutral,prompt_multi_term_zeroshot,Task: Analyze the sentiment of specific terms ...
1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,positive,prompt_multi_term_zeroshot,Task: Analyze the sentiment of specific terms ...
2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,neutral,prompt_multi_term_zeroshot,Task: Analyze the sentiment of specific terms ...
3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,neutral,prompt_multi_term_zeroshot,Task: Analyze the sentiment of specific terms ...
4,1,4,The server was so busy the night we visited th...,server,negative,negative,prompt_multi_term_zeroshot,Task: Analyze the sentiment of specific terms ...


In [43]:
df.to_csv(OUTPUT_PATH, index = True)